In [3]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report



train_df = pd.read_csv('cleaned_train_data.csv')
test_df = pd.read_csv('cleaned_test_data.csv')

selected_features = ['sending_bytes', 'min_packets_len','ttl_values_max', 'ttl_values_mean', 'distinct_A_records']
X_train = train_df[selected_features].values
y_train = train_df['label'].values  

X_test = test_df[selected_features].values
y_test = test_df['label'].values


X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.int32)

X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.int32)






In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

# Define input layer
input_layer = keras.Input(shape=(5,))  

# Simple Dense Layer with Regularization
x = layers.Dense(12, activation='relu', kernel_regularizer=regularizers.l2(0.001))(input_layer)

# Reshape for Multi-Head Attention
x = layers.Reshape((1, 12))(x)  

# Multi-Head Attention 
attn_output = layers.MultiHeadAttention(num_heads=1, key_dim=8)(x, x, x)  
attn_output = layers.LayerNormalization()(attn_output)

# Residual Connection 
x = layers.Add()([x, attn_output])

# Flatten & Dense Layers for Classification
x = layers.Flatten()(x)
x = layers.Dense(2, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.Dropout(0.3)(x)  

# Output Layer (Binary Classification)
output_layer = layers.Dense(2, activation='softmax')(x)

# Create Model
model = keras.Model(inputs=input_layer, outputs=output_layer)

# Compile Model with Optimizer & Loss
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print Model Summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 12)        │         72 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 1, 12)     │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 1, 12)     │        420 │ reshape_1[0][0],  │
│ (MultiHeadAttentio… │                   │            │ reshape_1[0][0],  │
│                     │                   │            │ reshape_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 1, 12)     │         24 │ multi_head_atten… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 1, 12)     │          0 │ reshape_1[0][0],  │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 12)        │          0 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 2)         │         26 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 2)         │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 2)         │          6 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 548 (2.14 KB)

 Trainable params: 548 (2.14 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=10, batch_size=32)

Epoch 1/10
26725/26725 ━━━━━━━━━━━━━━━━━━━━ 82s 3ms/step - accuracy: 0.8319 - loss: 0.3323 - val_accuracy: 0.9958 - val_loss: 0.0289
Epoch 2/10
26725/26725 ━━━━━━━━━━━━━━━━━━━━ 76s 3ms/step - accuracy: 0.8464 - loss: 0.2332 - val_accuracy: 0.9928 - val_loss: 0.0441
Epoch 3/10
26725/26725 ━━━━━━━━━━━━━━━━━━━━ 73s 3ms/step - accuracy: 0.8454 - loss: 0.2333 - val_accuracy: 0.9979 - val_loss: 0.0251
Epoch 4/10
26725/26725 ━━━━━━━━━━━━━━━━━━━━ 79s 3ms/step - accuracy: 0.8473 - loss: 0.2277 - val_accuracy: 0.9979 - val_loss: 0.0214
Epoch 5/10
26725/26725 ━━━━━━━━━━━━━━━━━━━━ 74s 3ms/step - accuracy: 0.8473 - loss: 0.2293 - val_accuracy: 0.9965 - val_loss: 0.0347
Epoch 6/10
26725/26725 ━━━━━━━━━━━━━━━━━━━━ 71s 3ms/step - accuracy: 0.8475 - loss: 0.2338 - val_accuracy: 0.9931 - val_loss: 0.0382
Epoch 7/10
26725/26725 ━━━━━━━━━━━━━━━━━━━━ 72s 3ms/step - accuracy: 0.8470 - loss: 0.2258 - val_accuracy: 0.9930 - val_loss: 0.0291
Epoch 8/10
26725/26725 ━━━━━━━━━━━━━━━━━━━━ 79s 3ms/step - accuracy: 

In [13]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

# Get predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels

# Convert true labels if they are one-hot encoded
y_true = np.argmax(y_test, axis=1) if len(y_test.shape) > 1 else y_test

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy*100:.2f}")

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot confusion matrix using Seaborn
print(conf_matrix)

6687/6687 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step
Accuracy: 98.40
[[103373   3418]
 [     0 107189]]


In [20]:
from sklearn.metrics import classification_report

# Get predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels

# Convert true labels if they are one-hot encoded
y_true = np.argmax(y_test, axis=1) if len(y_test.shape) > 1 else y_test

# Generate classification report
report = classification_report(y_true, y_pred)
print(report)


6687/6687 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step
              precision    recall  f1-score   support

           0       1.00      0.97      0.98    106791
           1       0.97      1.00      0.98    107189

    accuracy                           0.98    213980
   macro avg       0.98      0.98      0.98    213980
weighted avg       0.98      0.98      0.98    213980



In [16]:
df_ben=pd.read_csv("benign_1.csv")
df_ben = df_ben[test_df.columns]

# Display the first few rows to verify
print(df_ben.head())

                                             flow_id  \
0  2020-11-20 09:45:50.596673_192.168.20.38_65180...   
1  2020-11-20 09:34:36.652573_192.168.20.38_51436...   
2  2020-11-20 09:35:42.439182_192.168.20.38_64403...   
3  2020-11-20 09:36:29.117098_192.168.20.38_61986...   
4  2020-11-20 09:36:38.505992_192.168.20.38_56030...   

                    timestamp         src_ip  src_port   dst_ip  dst_port  \
0  2020-11-20 09:45:50.596673  192.168.20.38     65180  8.8.8.8        53   
1  2020-11-20 09:34:36.652573  192.168.20.38     51436  8.8.8.8        53   
2  2020-11-20 09:35:42.439182  192.168.20.38     64403  8.8.8.8        53   
3  2020-11-20 09:36:29.117098  192.168.20.38     61986  8.8.8.8        53   
4  2020-11-20 09:36:38.505992  192.168.20.38     56030  8.8.8.8        53   

   duration  total_bytes  receiving_bytes  sending_bytes  ...  \
0  0.160896          330              138            192  ...   
1  0.015870          152              152              0  ...   
2  0.

In [17]:
df_ben['label'] = 0



In [18]:
df_ben = df_ben[~df_ben.apply(lambda row: row.astype(str).str.contains("not a dns flow", case=False, na=False)).any(axis=1)]




print(f"Rows after removal: {df_ben.shape[0]}")

Rows after removal: 32716


In [26]:
from sklearn.preprocessing import StandardScaler

# Select only numerical columns
numerical_cols = df_ben.select_dtypes(include=['number']).columns

# Initialize scaler
scaler = StandardScaler()

# Fit and transform the numerical columns
df_ben[numerical_cols] = scaler.fit_transform(df_ben[numerical_cols])

# Display the first few rows to check
print(df_ben.head())


                                             flow_id  \
0  2020-11-20 09:45:50.596673_192.168.20.38_65180...   
1  2020-11-20 09:34:36.652573_192.168.20.38_51436...   
2  2020-11-20 09:35:42.439182_192.168.20.38_64403...   
3  2020-11-20 09:36:29.117098_192.168.20.38_61986...   
4  2020-11-20 09:36:38.505992_192.168.20.38_56030...   

                    timestamp         src_ip  src_port   dst_ip  dst_port  \
0  2020-11-20 09:45:50.596673  192.168.20.38  1.620219  8.8.8.8  -0.02065   
1  2020-11-20 09:34:36.652573  192.168.20.38 -1.214195  8.8.8.8  -0.02065   
2  2020-11-20 09:35:42.439182  192.168.20.38  1.459979  8.8.8.8  -0.02065   
3  2020-11-20 09:36:29.117098  192.168.20.38  0.961523  8.8.8.8  -0.02065   
4  2020-11-20 09:36:38.505992  192.168.20.38 -0.266778  8.8.8.8  -0.02065   

   duration  total_bytes  receiving_bytes  sending_bytes  ...  \
0 -0.010128     0.012908        -0.005154       1.759067  ...   
1 -0.011822    -0.012026        -0.003192      -0.860375  ...   
2 -0.

In [19]:
import numpy as np

selected_features =['sending_bytes', 'min_packets_len','ttl_values_max', 'ttl_values_mean', 'distinct_A_records']
label_column = 'label'  

X_test_new = df_ben[selected_features].values  # Features
y_test_new = df_ben[label_column].values       # Actual labels


# Convert to Tensor
X_test_ttensor = tf.convert_to_tensor(X_test_new, dtype=tf.float32)

# Predict class probabilities
predicted_probs = model.predict(X_test_ttensor)

# Get predicted class (0 or 1)
predicted_classes = np.argmax(predicted_probs, axis=1)

# Compare with actual labels
correct_predictions = np.sum(predicted_classes == y_test_new)
incorrect_predictions = np.sum(predicted_classes != y_test_new)

# Print results
print(f"Total Samples: {len(y_test_new)}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Incorrect Predictions: {incorrect_predictions}")
print(f"Accuracy: {correct_predictions / len(y_test_new) * 100:.2f}%")


1023/1023 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Total Samples: 32716
Correct Predictions: 31203
Incorrect Predictions: 1513
Accuracy: 95.38%


In [21]:
df_mal=pd.read_csv("heavy_text.csv")
df_mal = df_mal[test_df.columns]

# Display the first few rows to verify
print(df_mal.head())

                                             flow_id  \
0  2020-11-23 19:29:36.240593_192.168.20.38_54289...   
1  2020-11-23 19:28:24.070956_192.168.20.38_49856...   
2  2020-11-23 19:30:10.156905_192.168.20.38_63064...   
3  2020-11-23 19:28:19.459205_192.168.20.38_63545...   
4  2020-11-23 19:29:29.178753_192.168.20.38_50750...   

                    timestamp         src_ip  src_port         dst_ip  \
0  2020-11-23 19:29:36.240593  192.168.20.38     54289  192.168.20.72   
1  2020-11-23 19:28:24.070956  192.168.20.38     49856  192.168.20.72   
2  2020-11-23 19:30:10.156905  192.168.20.38     63064  192.168.20.72   
3  2020-11-23 19:28:19.459205  192.168.20.38     63545  192.168.20.72   
4  2020-11-23 19:29:29.178753  192.168.20.38     50750  192.168.20.72   

   dst_port  duration  total_bytes  receiving_bytes  sending_bytes  ...  \
0        53       0.0           96               96              0  ...   
1        53       0.0           96               96              0  ...   

C:\Users\Joselin Rachel\AppData\Local\Temp\ipykernel_17468\3195647395.py:1: DtypeWarning: Columns (87,88,92,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mal=pd.read_csv("heavy_text.csv")


In [23]:
df_mal['label'] = 1



In [24]:
df_mal = df_mal[~df_mal.apply(lambda row: row.astype(str).str.contains("not a dns flow", case=False, na=False)).any(axis=1)]

# Save the cleaned dataset


print(f"Rows after removal: {df_mal.shape[0]}")

Rows after removal: 27210


In [25]:
from sklearn.preprocessing import StandardScaler

# Select only numerical columns
numerical_cols = df_mal.select_dtypes(include=['number']).columns

# Initialize scaler
scaler = StandardScaler()

# Fit and transform the numerical columns
df_mal[numerical_cols] = scaler.fit_transform(df_mal[numerical_cols])

# Display the first few rows to check
print(df_mal.head())


                                             flow_id  \
0  2020-11-23 19:29:36.240593_192.168.20.38_54289...   
1  2020-11-23 19:28:24.070956_192.168.20.38_49856...   
2  2020-11-23 19:30:10.156905_192.168.20.38_63064...   
3  2020-11-23 19:28:19.459205_192.168.20.38_63545...   
4  2020-11-23 19:29:29.178753_192.168.20.38_50750...   

                    timestamp         src_ip  src_port         dst_ip  \
0  2020-11-23 19:29:36.240593  192.168.20.38 -0.478867  192.168.20.72   
1  2020-11-23 19:28:24.070956  192.168.20.38 -1.225674  192.168.20.72   
2  2020-11-23 19:30:10.156905  192.168.20.38  0.999415  192.168.20.72   
3  2020-11-23 19:28:19.459205  192.168.20.38  1.080447  192.168.20.72   
4  2020-11-23 19:29:29.178753  192.168.20.38 -1.075066  192.168.20.72   

   dst_port  duration  total_bytes  receiving_bytes  sending_bytes  ...  \
0 -0.062901 -0.006308    -0.006074        -0.006073      -0.013555  ...   
1 -0.062901 -0.006308    -0.006074        -0.006073      -0.013555  ...   

In [26]:
import numpy as np
# Define selected features (must match training features)
selected_features =['sending_bytes', 'min_packets_len','ttl_values_max', 'ttl_values_mean', 'distinct_A_records']
label_column = 'label'  # Change this to match the actual label column name
# Extract features and labels
X_test_new = df_mal[selected_features].values  # Features
y_test_new = df_mal[label_column].values       # Actual labels
#scaler = StandardScaler()
#X_test_scaled = scaler.fit_transform(X_test_new)

# Convert to Tensor
X_test_ttensor = tf.convert_to_tensor(X_test_new, dtype=tf.float32)

# Predict class probabilities
predicted_probs = model.predict(X_test_ttensor)

# Get predicted class (0 or 1)
predicted_classes = np.argmax(predicted_probs, axis=1)

# Compare with actual labels
correct_predictions = np.sum(predicted_classes == y_test_new)
incorrect_predictions = np.sum(predicted_classes != y_test_new)

# Print results
print(f"Total Samples: {len(y_test_new)}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Incorrect Predictions: {incorrect_predictions}")
print(f"Accuracy: {correct_predictions / len(y_test_new) * 100:.2f}%")


851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Total Samples: 27210
Correct Predictions: 27210
Incorrect Predictions: 0
Accuracy: 100.00%
